<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import category_encoders as ce

In [2]:
# Read and save the csv file
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Display the columns up to 500
pd.set_option('display.max_columns', 500)

# Check the 1st 5 rows
print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# Check for null values
df.isna().sum().sum()

0

In [4]:
# Check how many categorical variables we are dealing with
for i in df.columns[1:]:
    if df[i].dtype == object:
        print(df[i].value_counts(),'\n')

Male      3555
Female    3488
Name: gender, dtype: int64 

No     3641
Yes    3402
Name: Partner, dtype: int64 

No     4933
Yes    2110
Name: Dependents, dtype: int64 

Yes    6361
No      682
Name: PhoneService, dtype: int64 

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64 

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64 

No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64 

No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64 

No                     3095
Yes                    2422
No internet service    1526
Name: DeviceProtection, dtype: int64 

No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64 

No                     2810
Yes                    2707
No internet service    15

In [5]:
# Check the data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [6]:
# Change Integers data to float data
for i in df.columns[1:]:
    if (df[i].dtype == 'int64') or (df[i].dtype == int):
        df[i] = df[i].astype(float)

In [7]:
# Return a new df where TotalCharages column has no ' '
df = df[df['TotalCharges'] != ' ']

# Change to numeric data type
df['TotalCharges'] = df['TotalCharges'].astype(float)

In [8]:
# Check if the integers change to floats
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7032 non-null object
gender              7032 non-null object
SeniorCitizen       7032 non-null float64
Partner             7032 non-null object
Dependents          7032 non-null object
tenure              7032 non-null float64
PhoneService        7032 non-null object
MultipleLines       7032 non-null object
InternetService     7032 non-null object
OnlineSecurity      7032 non-null object
OnlineBackup        7032 non-null object
DeviceProtection    7032 non-null object
TechSupport         7032 non-null object
StreamingTV         7032 non-null object
StreamingMovies     7032 non-null object
Contract            7032 non-null object
PaperlessBilling    7032 non-null object
PaymentMethod       7032 non-null object
MonthlyCharges      7032 non-null float64
TotalCharges        7032 non-null float64
Churn               7032 non-null object
dtypes: float64(4), object(1

In [9]:
# Save object data type to cols. Will use for ordinal encoding.
cols = []
for i in df.columns[1:]:
    if df[i].dtype == object:
        cols.append(i)
print(cols)

['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']


In [10]:
# Ordinal Encoder
encoder = ce.OrdinalEncoder()

# Fit & Transform
df = encoder.fit_transform(df)

# Scale continuous_col
continuous_col = ['MonthlyCharges', 'TotalCharges']

# Use MinMaxScaler
scaled = MinMaxScaler()

# Save the Scaled values to their respective column
df[continuous_col] = scaled.fit_transform(df[continuous_col])

# Check first 5 rows
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,1,0.0,1,1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,0.115423,0.001275,1
1,2,2,0.0,2,1,34.0,2,2,1,2,2,2,1,1,1,2,2,2,0.385075,0.215867,1
2,3,2,0.0,2,1,2.0,2,2,1,2,1,1,1,1,1,1,1,2,0.354229,0.010310,2
3,4,2,0.0,2,1,45.0,1,1,1,2,2,2,2,1,1,2,2,3,0.239303,0.210241,1
4,5,1,0.0,2,1,2.0,2,2,2,1,2,1,1,1,1,1,1,1,0.521891,0.015330,2


# K-fold Cross Validation

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

seed = 42
np.random.seed(seed)

# Inputs- From 'gender' to 'TotalCharges'
X = df.iloc[:,1:-1].values

# Output- 'Churn'
y = df.iloc[:,-1].values

# 5-fold cross validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(X, y):
    # Create Model
    model = Sequential()
    # Inputs are the number of features for X, 16 node in hidden layers
    model.add(Dense(16, input_dim=X.shape[1], activation='relu'))
    # Hidden layers - 16 nodes
    model.add(Dense(16, activation='relu'))
    # Output layer 1 node
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fit the model
    model.fit(X[train], y[train], epochs=150, batch_size=10, verbose=0)
    
    # Evaluate model
    scores = model.evaluate(X[test], y[test], verbose=0)
    print(f'{model.metrics_names[1]}: {(scores[1]*100):.2f}%')
    cvscores.append(scores[1]*100)
print(f'{np.mean(cvscores):.2f}% +/- {np.std(cvscores):.2f}%')

acc: 73.42%
acc: 73.42%
acc: 73.42%
acc: 73.40%
acc: 73.45%
73.42% +/- 0.02%


# Hyperparameter Tuning with GridSearchCV

In [22]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

seed = 42
np.random.seed(seed)

# Inputs- From 'gender' to 'TotalCharges'
X = df.iloc[:,1:-1].values

# Output- 'Churn'
y = df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=seed)

def create_model(neurons=4, dropout_rate=0, optimizer='adam', activation='relu'):
    # Create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=X.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation=activation))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create Wrapper model
model = KerasClassifier(build_fn=create_model, verbose=1)

# Define grid search parameters
param_grid= {'batch_size': [32],
             'epochs': [100],
             'dropout_rate': [0],
             'neurons': [4, 16, 32],
             'optimizer': ['adam', 'rmsprop'],
             'activation': ['relu', 'sigmoid', 'tanh']
            }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Means: {mean}, Stdev: {stdev} with: {param}')

Epoch 1/100
3516/3516 [==============================] - 0s 117us/sample - loss: 9.5256 - acc: 0.2898
Epoch 2/100
3516/3516 [==============================] - 0s 48us/sample - loss: 3.9584 - acc: 0.5995
Epoch 3/100
3516/3516 [==============================] - 0s 54us/sample - loss: 1.7641 - acc: 0.6965
Epoch 4/100
3516/3516 [==============================] - 0s 50us/sample - loss: 1.6754 - acc: 0.7056
Epoch 5/100
3516/3516 [==============================] - 0s 63us/sample - loss: 1.6259 - acc: 0.7190
Epoch 6/100
3516/3516 [==============================] - 0s 53us/sample - loss: 1.5370 - acc: 0.7113
Epoch 7/100
3516/3516 [==============================] - 0s 53us/sample - loss: 2.1325 - acc: 0.7176
Epoch 8/100
3516/3516 [==============================] - 0s 48us/sample - loss: 1.7312 - acc: 0.7207
Epoch 9/100
3516/3516 [==============================] - 0s 57us/sample - loss: 1.3444 - acc: 0.6934
Epoch 10/100
3516/3516 [==============================] - 0s 48us/sample - loss: 1.5077 - 

In [23]:
from sklearn.metrics import accuracy_score

# Predict X_test
best = grid.best_estimator_
y_pred = best.predict(X_test)
print('Accuracy Score:', accuracy_score(y_test, y_pred))

1758/1758 [==============================] - 0s 250us/sample
Accuracy Score: 0.7872582480091013


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?